# Jeff Pinegar
Project 3: ETL 
Due Jan 30, 2023

### Extract and Transform Data Scienc Salary
---

In [1]:
# Import needed libraries
import pandas as pd     # if you get an error message module not found you need to add this to your environment.
import os               # this tells your project about the OS of your machine.  This will let you create machine independent notebooks
import numpy as np      # loads in the math tools
from datetime import datetime

---
### Extract

In [3]:
# Import data file
# My Jupyter notebook is in the current directory " . "
# There is a sub directory "Resources" and the file "IPC shipped from Germany by serial number 2022 (until 5.12.2022).csv" is located there.
salary_data = os.path.join('.','data', 'DS_salary.csv')            

# Opens the CVS file and read it in as a panda dataframe then print the head.
df_raw = pd.read_csv(salary_data, index_col=0, encoding="utf-8")                    
df_raw.head(2)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_location_3Char,comp_size,Experience,Emply_Type,Country_Company,Country_Emply,Company_size,Category,Remote_Work,Machine_learning
id,,,,,,,,,,,,,,,,,,,,
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,DEU,L,Mid-level Intermediate,Full-time,Germany,Germany,Large_Greater_than_250_Emp,Scientist,0-20%,Not ML
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,JPN,S,Senior-level Expert,Full-time,Japan,Japan,Small_Less_than_50_Emp,Scientist,0-20%,ML Professional


# Create Data for website

In [4]:
# export data as an HTML table that can be added to the web page.
dfs = df_raw
file_one = os.path.join('.','data', 'DS_salary.html')
dfs.to_html(file_one, classes='table table-stripped')

NameError: name 'df3' is not defined